In [43]:
import glob
import pandas as pd

import h2o
from h2o.automl import H2OAutoML

import pandas as pd
import numpy as np

import matplotlib as plt
%matplotlib inline

#Import the Estimators
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators import H2OXGBoostEstimator

#Import h2o grid search 
import h2o.grid 
from h2o.grid.grid_search import H2OGridSearch

xgboostModel = h2o.estimators.xgboost.H2OXGBoostEstimator()
#H2OXGBoostEstimator.available()

# xgboostModel = h2o.estimators.xgboost.H2OXGBoostEstimator()
# H2OXGBoostEstimator.available()

from datetime import datetime


In [44]:
challenge_set = pd.read_csv("/mnt/SMB_share/mark/flight_competition/challenge_set.csv")
challenge_set = challenge_set[["flight_id", "tow"]].drop_duplicates()


In [45]:
files = glob.glob("data/by_type_consolidated/*.csv")
segments = [x.split("/")[-1].replace(".csv", "") for x in files]
segments

['B39M',
 'A21N',
 'A343',
 'A320',
 'B752',
 'BCS3',
 'A319',
 'E190',
 'E195',
 'A321',
 'B738',
 'AT76',
 'A20N',
 'B739',
 'E290',
 'A332',
 'CRJ9',
 'B763',
 'A333',
 'B788',
 'A310',
 'A359',
 'C56X',
 'B789',
 'B38M',
 'BCS1',
 'B737',
 'B772',
 'B773',
 'B77W']

In [46]:
h2o.init(max_mem_size = "25g", nthreads = 12)


Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,8 hours 58 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,3 months and 12 days
H2O_cluster_name:,H2O_from_python_mark_eaztjx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20.68 Gb
H2O_cluster_total_cores:,128
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [47]:

# test_data = pd.read_csv("data/by_type_consolidated/B39M.csv")
# test_data


In [48]:
done = ["B39M", "A21N", "A343", "B752", "BCS3", "A319", "E190", "E195", "A321", "A320", "B738", "AT76", "A20N", "B739", "A332", "CRJ9", "B763", "A333", "B788", "A310", "A359", "B789", "B38M", "BCS1", "B737", "B772", "B77W"]
#A320, do this later
segments = [seg for seg in segments if seg not in done]

In [49]:
for segment in segments: 
    print(segment)
    try:
        this_data = h2o.import_file("data/by_type_consolidated/"+segment+".csv")
        mask = this_data["tow"].isna()
        this_data = this_data[~mask,:]
        
        x = this_data.columns
        y = "tow"
        ignore = ["segment", "fold_group", "aircraft_type", "Unnamed: 0"] 
        x = list(set(x) - set(ignore))
        x.remove(y)

        test, train = this_data.split_frame(ratios = [0.1], seed = 1)
        print(len(this_data))
        aml = H2OAutoML(max_models=40, seed=10, stopping_metric="RMSE")
        #, verbosity="info"
        
        aml.train(x=x, y=y, training_frame=train, fold_column="fold_group")
        leader_model = aml.leader
        
        now = datetime.now()
        perform = round(leader_model.model_performance(test).rmse(), 0)
        h2o.save_model(model=leader_model, path="data/models/state_2_"+str(segment)+"_"+now.strftime("%m_%d_%y")+"_"+str(perform))
        print(round(leader_model.rmse(), 0))
        print(perform)
    
    except Exception as e: 
        print(e)

E290
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
46
AutoML progress: |
23:05:13.688: Fold column fold_group will be used for cross-validation. nfolds parameter will be ignored.
23:05:13.704: _fold_column param, Invalid fold column 'fold_group', fold cannot be constant
23:05:13.704: _train param, Dropping bad and constant columns: [flight_id]
23:05:13.705: _fold_column param, Invalid fold column 'fold_group', fold cannot be constant
23:05:13.705: _response param, Response cannot be constant.
23:05:13.711: _fold_column param, Invalid fold column 'fold_group', fold cannot be constant
23:05:13.711: _train param, Dropping bad and constant columns: [flight_id]
23:05:13.711: _fold_column param, Invalid fold column 'fold_group', fold cannot be constant
23:05:13.712: _response param, Response cannot be constant.
23:05:13.716: _fold_column param, Invalid fold column 'fold_group', fold cannot be constant
23:05:13.716: _train param, Dropping bad a